<a href="https://colab.research.google.com/github/samantha8990/finalproject/blob/main/Machine%20Learning/Gender_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 198 kB 66.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=3fa72fbde04231557fd6dbe6a595bea62114ef730b7358cc37135c74e0637da1
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkDates").getOrCreate()

In [3]:
# Load in data
from pyspark import SparkFiles
url ="https://murderanalytics.s3.us-west-1.amazonaws.com/CleanData.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("CleanData.csv"), sep=",", header=True, inferSchema=True)
df.show()

+---+-------------+------+----+--------+--------------------+------+------+------+-------+--------------------+--------------------+--------------------+--------------------+
|_c0|     CNTYFIPS|Solved|Year|   Month|            Homicide|VicAge|VicSex|OffAge| OffSex|              Weapon|        Relationship|        Circumstance|           Situation|
+---+-------------+------+----+--------+--------------------+------+------+------+-------+--------------------+--------------------+--------------------+--------------------+
|  0|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    40|Female|    15|   Male|Firearm, type not...|              Sister|Circumstances und...|Multiple victims/...|
|  1|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    18|Female|    15|   Male|Firearm, type not...|              Sister|Circumstances und...|Multiple victims/...|
|  2|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    49|  Male|    34|   Male|Firearm, type not...|            St

Data Pre-Processing

In [4]:
import pandas as pd
data = df.toPandas()
data = data[data.OffSex != "Unknown"]
data.head()

,_c0,CNTYFIPS,Solved,Year,Month,Homicide,VicAge,VicSex,OffAge,OffSex,Weapon,Relationship,Circumstance,Situation
0,0,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,40,Female,15,Male,"Firearm, type not stated",Sister,Circumstances undetermined,Multiple victims/single offender
1,1,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,18,Female,15,Male,"Firearm, type not stated",Sister,Circumstances undetermined,Multiple victims/single offender
2,2,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,49,Male,34,Male,"Firearm, type not stated",Stranger,Felon killed by police,Single victim/multiple offenders
3,3,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,32,Female,33,Male,Other or type unknown,Wife,Circumstances undetermined,Single victim/single offender
6,6,"Anchorage, AK",Yes,2016,January,Manslaughter by negligence,2,Male,21,Female,"Handgun - pistol, revolver, etc",Son,Children playing with gun,Single victim/single offender


In [5]:
MLData = data[["CNTYFIPS","Solved", "Month", "VicAge", "OffSex", "Weapon", "Relationship"]]
MLData.head()

,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son


Using Machine Learning we will learn how you are going to DIE.



In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Create numerical encoding for each unique column

In [7]:
#Solved
solved = np.unique(MLData['Solved'])
solved

array(['No', 'Yes'], dtype=object)

In [8]:
sle = LabelEncoder()
solved_labels = sle.fit_transform(MLData['Solved'])
solved_mappings = {index: label for index, label in 
                  enumerate(sle.classes_)}
solved_mappings

{0: 'No', 1: 'Yes'}

In [9]:
MLData['SolvedLabel'] = solved_labels
MLData.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1


In [10]:
#CNTYFIPS encoding
location = np.unique(MLData['CNTYFIPS'])
location


array(['Abbeville, SC', 'Acadia, LA', 'Accomack, VA', ..., 'Young, TX',
       'Yuba, CA', 'Yuma, AZ'], dtype=object)

In [11]:
cle = LabelEncoder()
location_labels = cle.fit_transform(MLData['CNTYFIPS'])
location_mappings = {index: label for index, label in 
                  enumerate(cle.classes_)}
location_mappings

{0: 'Abbeville, SC',
 1: 'Acadia, LA',
 2: 'Accomack, VA',
 3: 'Ada, ID',
 4: 'Adair, KY',
 5: 'Adair, MO',
 6: 'Adair, OK',
 7: 'Adams, CO',
 8: 'Adams, IL',
 9: 'Adams, NE',
 10: 'Adams, PA',
 11: 'Adams, WA',
 12: 'Adams, WI',
 13: 'Addison, VT',
 14: 'Aiken, SC',
 15: 'Aitkin, MN',
 16: 'Alachua, FL',
 17: 'Alamance, NC',
 18: 'Alameda, CA',
 19: 'Alamosa, CO',
 20: 'Albany, NY',
 21: 'Albany, WY',
 22: 'Albemarle, VA',
 23: 'Alexander, NC',
 24: 'Alexandria, VA',
 25: 'Alfalfa, OK',
 26: 'Alger, MI',
 27: 'Allegan, MI',
 28: 'Allegany, MD',
 29: 'Allegany, NY',
 30: 'Allegheny, PA',
 31: 'Allen, IN',
 32: 'Allen, KS',
 33: 'Allen, KY',
 34: 'Allen, LA',
 35: 'Allen, OH',
 36: 'Allendale, SC',
 37: 'Alpena, MI',
 38: 'Amador, CA',
 39: 'Amelia, VA',
 40: 'Amherst, VA',
 41: 'Anchorage, AK',
 42: 'Anderson, KS',
 43: 'Anderson, SC',
 44: 'Anderson, TN',
 45: 'Anderson, TX',
 46: 'Andrew, MO',
 47: 'Andrews, TX',
 48: 'Androscoggin, ME',
 49: 'Angelina, TX',
 50: 'Anne Arundel, MD',


In [12]:
MLData['LocationLabel'] = location_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,41
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,41
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,41
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,41
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,41
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,41
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,41
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1,41
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1,41
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1,41


In [13]:
#Month encoding
Month = np.unique(MLData['Month'])
Month


array(['April', 'August', 'December', 'February', 'January', 'July',
       'June', 'March', 'May', 'November', 'October', 'September'],
      dtype=object)

In [14]:
mle = LabelEncoder()
month_labels = mle.fit_transform(MLData['Month'])
month_mappings = {index: label for index, label in 
                  enumerate(mle.classes_)}
month_mappings

{0: 'April',
 1: 'August',
 2: 'December',
 3: 'February',
 4: 'January',
 5: 'July',
 6: 'June',
 7: 'March',
 8: 'May',
 9: 'November',
 10: 'October',
 11: 'September'}

In [15]:
MLData['MonthLabel'] = month_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,41,4
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,41,4
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,41,4
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,41,4
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,41,4
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,41,3
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,41,7
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1,41,8
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1,41,8
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1,41,6


skipping VicAge due to it already being a numerical value

In [16]:
#OffSex encoding
Gender = np.unique(MLData['OffSex'])
Gender

array(['Female', 'Male'], dtype=object)

In [17]:
gle = LabelEncoder()
gender_labels = gle.fit_transform(MLData['OffSex'])
gender_mappings = {index: label for index, label in 
                  enumerate(gle.classes_)}
gender_mappings

{0: 'Female', 1: 'Male'}

In [18]:
MLData['GenderLabel'] = gender_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,41,4,1
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,41,4,1
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,41,4,1
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,41,4,1
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,41,4,0
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,41,3,1
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,41,7,1
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1,41,8,1
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1,41,8,1
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1,41,6,1


In [19]:
#Weapon encoding
Weapon = np.unique(MLData['Weapon'])
Weapon

array(['Asphyxiation - includes death by gas',
       'Blunt object - hammer, club, etc', 'Drowning', 'Explosives',
       'Fire', 'Firearm, type not stated',
       'Handgun - pistol, revolver, etc', 'Knife or cutting instrument',
       'Narcotics or drugs, sleeping pills', 'Other gun',
       'Other or type unknown', 'Personal weapons, includes beating',
       'Poison - does not include gas', 'Pushed or thrown out window',
       'Rifle', 'Shotgun', 'Strangulation - hanging'], dtype=object)

In [20]:
wle = LabelEncoder()
weapon_labels = wle.fit_transform(MLData['Weapon'])
weapon_mappings = {index: label for index, label in 
                  enumerate(wle.classes_)}
weapon_mappings

{0: 'Asphyxiation - includes death by gas',
 1: 'Blunt object - hammer, club, etc',
 2: 'Drowning',
 3: 'Explosives',
 4: 'Fire',
 5: 'Firearm, type not stated',
 6: 'Handgun - pistol, revolver, etc',
 7: 'Knife or cutting instrument',
 8: 'Narcotics or drugs, sleeping pills',
 9: 'Other gun',
 10: 'Other or type unknown',
 11: 'Personal weapons, includes beating',
 12: 'Poison - does not include gas',
 13: 'Pushed or thrown out window',
 14: 'Rifle',
 15: 'Shotgun',
 16: 'Strangulation - hanging'}

In [21]:
MLData['WeaponLabel'] = weapon_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,41,4,1,5
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,41,4,1,5
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,41,4,1,5
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,41,4,1,10
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,41,4,0,6
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,41,3,1,5
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,41,7,1,5
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1,41,8,1,5
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1,41,8,1,16
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1,41,6,1,5


In [22]:
#Relationship encoding
Relationship = np.unique(MLData['Relationship'])
Relationship

array(['Acquaintance', 'Boyfriend', 'Brother', 'Common-law husband',
       'Common-law wife', 'Daughter', 'Employee', 'Employer',
       'Ex-husband', 'Ex-wife', 'Father', 'Friend', 'Girlfriend',
       'Homosexual relationship', 'Husband', 'In-law', 'Mother',
       'Neighbor', 'Other - known to victim', 'Other family',
       'Relationship not determined', 'Sister', 'Son', 'Stepdaughter',
       'Stepfather', 'Stepmother', 'Stepson', 'Stranger', 'Wife'],
      dtype=object)

In [23]:
rle = LabelEncoder()
relationship_labels = rle.fit_transform(MLData['Relationship'])
relationship_mappings = {index: label for index, label in 
                  enumerate(rle.classes_)}
relationship_mappings

{0: 'Acquaintance',
 1: 'Boyfriend',
 2: 'Brother',
 3: 'Common-law husband',
 4: 'Common-law wife',
 5: 'Daughter',
 6: 'Employee',
 7: 'Employer',
 8: 'Ex-husband',
 9: 'Ex-wife',
 10: 'Father',
 11: 'Friend',
 12: 'Girlfriend',
 13: 'Homosexual relationship',
 14: 'Husband',
 15: 'In-law',
 16: 'Mother',
 17: 'Neighbor',
 18: 'Other - known to victim',
 19: 'Other family',
 20: 'Relationship not determined',
 21: 'Sister',
 22: 'Son',
 23: 'Stepdaughter',
 24: 'Stepfather',
 25: 'Stepmother',
 26: 'Stepson',
 27: 'Stranger',
 28: 'Wife'}

In [24]:
MLData['RelationshipLabel'] = relationship_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel,RelationshipLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,41,4,1,5,21
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,41,4,1,5,21
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,41,4,1,5,27
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,41,4,1,10,28
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,41,4,0,6,22
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,41,3,1,5,27
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,41,7,1,5,11
10,"Anchorage, AK",Yes,May,19,Male,"Firearm, type not stated",Stranger,1,41,8,1,5,27
11,"Anchorage, AK",Yes,May,48,Male,Strangulation - hanging,Girlfriend,1,41,8,1,16,12
12,"Anchorage, AK",Yes,June,30,Male,"Firearm, type not stated",Acquaintance,1,41,6,1,5,0


In [25]:
#Split Dataframe to encoded/non encoded
encoded_DF = MLData[["SolvedLabel", "LocationLabel", "MonthLabel", "GenderLabel", "WeaponLabel", "RelationshipLabel"]]
encoded_DF.head()

,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel,RelationshipLabel
0,1,41,4,1,5,21
1,1,41,4,1,5,21
2,1,41,4,1,5,27
3,1,41,4,1,10,28
6,1,41,4,0,6,22


Which Gender is most likely to murder you?

In [26]:
X = encoded_DF.drop("GenderLabel", axis=1)
y = encoded_DF["GenderLabel"]
print(X.shape, y.shape)

(47416, 5) (47416,)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [28]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [30]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [31]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [32]:
 # Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 10,902
Trainable params: 10,902
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
1112/1112 - 2s - loss: 0.3312 - accuracy: 0.8950
Epoch 2/60
1112/1112 - 1s - loss: 0.3237 - accuracy: 0.8957
Epoch 3/60
1112/1112 - 1s - loss: 0.3128 - accuracy: 0.8958
Epoch 4/60
1112/1112 - 1s - loss: 0.3010 - accuracy: 0.8998
Epoch 5/60
1112/1112 - 1s - loss: 0.2975 - accuracy: 0.9018
Epoch 6/60
1112/1112 - 1s - loss: 0.2963 - accuracy: 0.9023
Epoch 7/60
1112/1112 - 1s - loss: 0.2952 - accuracy: 0.9035
Epoch 8/60
1112/1112 - 1s - loss: 0.2944 - accuracy: 0.9040
Epoch 9/60
1112/1112 - 1s - loss: 0.2932 - accuracy: 0.9046
Epoch 10/60
1112/1112 - 1s - loss: 0.2924 - accuracy: 0.9045
Epoch 11/60
1112/1112 - 1s - loss: 0.2918 - accuracy: 0.9053
Epoch 12/60
1112/1112 - 1s - loss: 0.2907 - accuracy: 0.9058
Epoch 13/60
1112/1112 - 1s - loss: 0.2903 - accuracy: 0.9061
Epoch 14/60
1112/1112 - 1s - loss: 0.2894 - accuracy: 0.9063
Epoch 15/60
1112/1112 - 1s - loss: 0.2891 - accuracy: 0.9068
Epoch 16/60
1112/1112 - 1s - loss: 0.2885 - accuracy: 0.9070
Epoch 17/60
1112/1112 - 1s - loss

In [35]:
#Quantify our trained Gender Model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


371/371 - 0s - loss: 0.2651 - accuracy: 0.9132
Normal Neural Network - Loss: 0.26506975293159485, Accuracy: 0.9131938815116882


In [36]:
#Make Prediction for the Gender of our Murderer
encoded_predictions = np.argmax(model.predict(X_test_scaled[:5]), axis = -1)
prediction_labels = gle.inverse_transform(encoded_predictions)

In [37]:
#Gender map reminder
print(gender_mappings)

{0: 'Female', 1: 'Male'}


In [38]:
print(f"Predicted Gender: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted Gender: ['Male' 'Female' 'Male' 'Male' 'Male']
Actual Labels: [1, 0, 1, 1, 1]
